In [25]:
import requests
import base64
import json
import pandas as pd
import sqlalchemy as db

# Fakturoid API credentials and parameters
FAKTUROID_API_BASE_URL = 'https://app.fakturoid.cz/api/v2/accounts'
FAKTUROID_SLUG = 'datadrivenmarketingdev'
FAKTUROID_USERNAME = 'jan.kadlecek+dev@marketing.bi'
FAKTUROID_API_KEY = '31f47e8632bd4b8e77f3dc7b19c7e2c945c3108b'

file_name = 'expenses'

# Form the complete URL with the slug
FAKTUROID_API_URL = f'{FAKTUROID_API_BASE_URL}/{FAKTUROID_SLUG}/{file_name}.json'

credentials = f"{FAKTUROID_USERNAME}:{FAKTUROID_API_KEY}"
credentials_encoded = base64.b64encode(credentials.encode()).decode('utf-8')
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {credentials_encoded}'
}

# Fetch invoices from Fakturoid API with slug parameter

# Define a function, that will accept the URL as an argument, performs the API requests, checks for presence of 'Link' in the response.headers and gets and appends all the data.
def fetch_data(url):
    all_data = []

    while True:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        all_data.extend(data)

        # Check for pagination
        if 'Link' in response.headers:
            links = requests.utils.parse_header_links(response.headers['Link'])
            next_url = None
            for link in links:
                if link['rel'] == 'next':
                    next_url = link['url']
                    break

            if next_url:
                url = next_url
            else:
                break
        else:
            break

    return all_data

# Now just run that function:
result = fetch_data(FAKTUROID_API_URL)

# Here we can test print the data, but I rather do it after I convert them to a Pandas DF.
# print(result)

In [9]:
#print(response.headers) #uncomment to take a look on the API response headers

In [28]:
#print(result[0].keys()) #uncomment to take a look at the API body structure

In [26]:
# PostgreSQL connection parameters
PG_PARAMS = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres",
    "port": "5432"
}

connection_string = f"postgresql://{PG_PARAMS['user']}:{PG_PARAMS['password']}@{PG_PARAMS['host']}:{PG_PARAMS['port']}/{PG_PARAMS['dbname']}"

df = pd.DataFrame(result)

#df.to_csv(f'{file_name}.csv', index=False) #uncomment to be able to check the table structure in this env

In [ ]:
# This outputs the dataframe to an SQL table in our PG db running in the docker.

df.to_sql(file_name + '_raw', connection_string, dtype={'lines': db.types.JSON}, if_exists='replace', index=False)